# Fitting a FLMP
In this notebook, we will fit a Fuzzy Logical Model of Perception (FLMP) to audiovisual speech perception data.

In [2]:
import numpy as np
from os import path
from glob import glob

import seaborn as sns
import matplotlib.pyplot as plt
sns.set()

from scipy.stats import binom
from scipy.optimize import minimize, LinearConstraint, Bounds

## Prepare Data
The data consists of five text files, each containing seven rows and five columns, where:
- Row 1: Audiotorial data
- Row 2: Visual data
- Rows 3-7: Audiovisual data
    - a combination of rows 1 and 2
    - visual goes from 'b' (row 3) to 'd' (row 7) 
    - audio goes from 'b' (col 1) to 'd' (col 5)

In [4]:
# get paths to data files
file_paths = glob("./data/*.txt")

# load all data into a single array
data = np.array([np.loadtxt(fname) for fname in file_paths])
N, M, K = data.shape

# define number of samples for each subject
n_samples = 24 

# split into the three different data types
data_A = data[:, 0, :]
data_V = data[:, 1, :]
data_AV = data[:, 2:, :]

## Fit FLMP

In [12]:
# create softmax function
def softmax(x):
    x = np.concatenate([
        np.array([x]).flatten(), [0]
    ])
    e = np.exp(x)
    return e[0] / e.sum()

In [19]:
def objective_function(theta, data, n_samples=24):
    # extract audio and visual parameters
    theta_A = theta[0:K]
    theta_V = theta[K: ]

    # get probabilities for audio and visual
    p_A = np.array([softmax(t) for t in theta_A]).reshape(-1,1)
    p_V = np.array([softmax(t) for t in theta_V]).reshape(-1,1)
    
    # compute audiovisual probabilities by
    # taking the outer product for all combinations of audio and visual
    p_AV = (p_A @ p_V.T) / (p_A @ p_V.T + (1 - p_A) @ (1 - p_V).T)

    # compute the log-likelihoods
    L_A = binom.logpmf(data[0], n_samples, p_A).flatten()
    L_V = binom.logpmf(data[1], n_samples, p_V).flatten()
    L_AV = binom.logpmf(data[2:], n_samples, p_AV).flatten()
    L = np.concatenate([L_A, L_V, L_AV]).sum()

    # return the negative log-likelihood
    return -L

In [21]:
def flmp_fit(data, n_samples):
    """Perform FLMP fit to data"""
    theta = np.zeros(K*2)
    opt_result = minimize(objective_function, theta, args=(data, n_samples))
    objective, theta_A, theta_V, success = (
        opt_result.fun, 
        (opt_result.x[0:K]), 
        (opt_result.x[K:]), 
        opt_result.success
    )
    return objective, theta_A, theta_V

In [22]:
# perform FLMP fit for each subject
theta_A = np.zeros((K, N))
theta_V = np.zeros((K, N))
neg_L = np.zeros(K)
for i in range(K):
    obj, tA, tV = flmp_fit(data[i], n_samples)

    # print results
    print(f'\nResults for subject {i+1}')
    print(f'Negative log-likelihood: {obj}')
    print("Audio:", tA)
    print("Visual:", tV)
    neg_L[i] = obj
    theta_A[i, :] = tA
    theta_V[i, :] = tV


Results for subject 1
Negative log-likelihood: 759.176649778559
Audio: [-1.30064673 -0.58587845 -0.03341137  0.15842406  0.2175664 ]
Visual: [0.3172988  0.63992309 0.86817301 1.56726399 1.73585165]

Results for subject 2
Negative log-likelihood: 745.9722540530329
Audio: [-1.39272991 -0.29460939  0.25587803  0.59689654  0.66610558]
Visual: [0.39729936 0.8235022  1.08317759 1.32397916 1.59814042]

Results for subject 3
Negative log-likelihood: 657.7838496588919
Audio: [-0.63204668 -0.30334678  0.06146461  0.26914399  0.36668493]
Visual: [-0.06229031  0.15831717  0.59669424  1.40463173  1.71854354]

Results for subject 4
Negative log-likelihood: 742.049544918451
Audio: [-0.65235525 -0.49645715 -0.38296974 -0.21665941  0.03682489]
Visual: [-0.89757279 -0.74897411 -0.51324393  0.72452223  1.13194639]

Results for subject 5
Negative log-likelihood: 805.7723566088321
Audio: [-0.84433922 -0.49486813 -0.3632717  -0.19551532 -0.08375168]
Visual: [-0.32344638 -0.14628076  0.55624044  1.58036762 